# Deploying a Model using Azure Machine Learning

The steps to deploy any model are:

1. Register the model
2. Prepare an entry script
3. Prepare an inference configuration and a deployment configuration
4. Deploy the model locally to ensure everything works
5. Choose a compute target.
6. Re-deploy the model to the cloud
7. Test the resulting web service.


[You can learn more by reading these official docs](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-and-where)

## 0. Connect to AML

In [1]:
import azureml.core
from azureml.core import Workspace, Dataset, Model, Environment
from azureml.core.model import InferenceConfig

subscription_id = 'ad2a181b-b804-4179-904a-012445b7d1f5'
resource_group = 'analyticsf7fa8b0c'
workspace_name = 'SignalBoxMLDev'

ws = Workspace(subscription_id, resource_group, workspace_name)

print('Connected to workspace:', ws.name)

model_version = 'product-1'
tags = {
    'source': 'tutorial',
    'production': False,
    'version': model_version
}

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


Connected to workspace: SignalBoxMLDev


## 1. Register a model

A typical situation for a deployed machine learning service is that you need the following components:

* resources representing the specific model that you want deployed (for example: a pytorch model file)
* code that you will be running in th service, that executes the model on a given input

Azure Machine Learning allows you to separate the deployment into two separate components, so that you can keep the same code, but merely update the model. We define the mechanism by which you upload a model separately from your code as "registering the model".

You can register a model by providing the local path of the model. You can provide the path of either a folder or a single file on your local machine.

In [2]:
# define a simple file as the specific 'model' we're going to use.
import pickle

model_parameters = {'name': 'product-recommender-tutorial-model',
        'parameters': {
            'weights': [0.6, 0.3, 0.1]
        }
    }
model_filename = 'product_recommender_tutorial.pkl'

with open(model_filename, 'wb') as handle:
    pickle.dump(model_parameters, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [3]:
# register the model
model_properties = {
    'source': 'product-recommender-tutorial',
    'version': model_version
}
model = Model.register(ws, 
                       model_name=model_parameters['name'], 
                       model_path=model_filename, 
                       tags=tags
                      )

Registering model product-recommender-tutorial-model


In [4]:
print(model)

Model(workspace=Workspace.create(name='SignalBoxMLDev', subscription_id='ad2a181b-b804-4179-904a-012445b7d1f5', resource_group='analyticsf7fa8b0c'), name=product-recommender-tutorial-model, id=product-recommender-tutorial-model:1, version=1, tags={'source': 'tutorial', 'production': 'False', 'version': 'product-1'}, properties={})


## 2. Prepare an entry script

The entry script receives data submitted to a deployed web service and passes it to the model. It then returns the model's response to the client. The script is specific to your model. The entry script must understand the data that the model expects and returns.

> You can use the environment variable `AZUREML_MODEL_DIR` to locate your model that you registered earlier

In [20]:
%%writefile product_recommender_source_dir/entry.py 


# this is an actual entry script
import json
import pickle
import random
import os

possible_outcomes = [
    {
        'productId': 5,
    },
    {
        'productId': 6,
    },
    {
        'productId': 7,
    },
]
def init():
    global model_parameters
    with open(os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'product_recommender_tutorial.pkl'), 'rb') as handle:
        model_parameters = pickle.load(handle)['parameters']

# request should have: payload: {},  version: str
def run(request):
    print(request)
    body = json.loads(request)
    version = body['version']
    ## check we can handle the version
    print(f'Version: {version}')

    # Run inference
    recommendation = recommend(body['payload'])
    print('recommendation:', recommendation)
   
    return recommendation

# assuming this is a paremeter-set recommender
# payload is json with arguments, parameters (both objects)
def recommend(r):
    # sort the offers by ID
    arguments = r['arguments']
    commonUserId = r['commonUserId']
    print(arguments)
    # you can load parameters from the model if you need ot.
    print('model params:', model_parameters['weights'])
    # random.choices() returns a list of k length
    return random.choices(possible_outcomes, model_parameters['weights'], k=1)[0] 



Overwriting product_recommender_source_dir/entry.py


## 3. Prepare an inference configuration and a deployment configuration

### Inference

An inference configuration describes the Docker container and files to use when initializing your web service. All of the files within your source directory, including subdirectories, will be zipped up and uploaded to the cloud when you deploy your web service.

### Deployment

A deployment configuration specifies the amount of memory and cores to reserve for your webservice will require in order to run, as well as configuration details of the underlying webservice. For example, a deployment configuration lets you specify that your service needs 2 gigabytes of memory, 2 CPU cores, 1 GPU core, and that you want to enable autoscaling.

In [16]:
env = Environment.from_pip_requirements(name='tutorial_environment', file_path="./model_requirements.txt")
inference_config = InferenceConfig(environment=env, source_directory='product_recommender_source_dir', entry_script='./entry.py')

In [17]:
# this creates a local webservice
from azureml.core.webservice import LocalWebservice
deployment_config = LocalWebservice.deploy_configuration(port=6789)

## 4. Deploy the model locally to ensure everything works

This part needs docker installed and running

In [18]:
service = Model.deploy(ws, "tutorial-service-local", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output=True)
print(service.get_logs())

Generating Docker build context.
Package creation Succeeded
Logging into Docker registry mlresgistryfe091fe9.azurecr.io
Logging into Docker registry mlresgistryfe091fe9.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM mlresgistryfe091fe9.azurecr.io/azureml/azureml_984ae42c92ab7e15d7808a45a9ac459f
 ---> 2f5da5597264
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> 0e7633d8ee18
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6ImFkMmExODFiLWI4MDQtNDE3OS05MDRhLTAxMjQ0NWI3ZDFmNSIsInJlc291cmNlR3JvdXBOYW1lIjoiYW5hbHl0aWNzZjdmYThiMGMiLCJhY2NvdW50TmFtZSI6InNpZ25hbGJveG1sZGV2Iiwid29ya3NwYWNlSWQiOiIyNTJmNTM5Ni03NmYwLTRjZDYtOGE3OS0wYjQ5ZGI3NmM1MGIifSwibW9kZWxzIjp7fSwibW9kZWxzSW5mbyI6e319 | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in 9b1fa4c30901
 ---> 61b97550fcd1
Step 4/5 : RUN mv '/var/azureml-app/tmpkeux4vlt.py' /var/azureml-app/main.py
 ---> Running in 3f4c972232b9
 ---> e6f5b75b22f1
Step 5/5 :

Call the local docker container to check that it works

In [8]:
data = {
    "version": "parameter-set-1", # check this version mathes
    "payload": {
        "commonUserId": "1234",
        "arguments": {
            "one": 1,
            "two": "foobar"
        }
    }
}

In [19]:
# check the model works
import requests
import json

uri = service.scoring_uri
print(uri)
# requests.get('http://localhost:6789')
headers = {'Content-Type': 'application/json'}
data_stringified = json.dumps(data)
response = requests.post(uri, data=data_stringified, headers=headers)
print('model responded with:')
print(response.json())

http://localhost:6789/score
model responded with:
{'productId': 6}


## 5. Choose a compute target.

[Learn more about choosing a target](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-and-where?tabs=python#choose-a-compute-target)

Options are:
* Local web service: for testing and debugging
* Azure Kubernetes Services: High scale production (probably don't do this without bigger discussion)
* Azure Container Instances: Low scale, less than 48GB RAM
* Azure Machine Learning compute cluster: best for batch inferencing


## 6. Re-deploy the model to the cloud

Deploy to an Azure Container Instanec

In [14]:
from azureml.core.webservice import AciWebservice
aci_deployment_config = AciWebservice.deploy_configuration(cpu_cores = 0.5, 
                                                           memory_gb = 1, 
                                                           tags=tags, 
                                                           auth_enabled=True)

In [15]:
# this cell creates a new ACI deployed into Azure.
service = Model.deploy(ws, 
                       "product-recommender-tutorial-aci", 
                       [model], 
                       inference_config, 
                       aci_deployment_config)

service.wait_for_deployment(show_output=True)
print(service.get_logs())

WebserviceException: WebserviceException:
	Message: Service product-recommender-tutorial-aci with the same name already exists, please use a different service name or delete the existing service.
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Service product-recommender-tutorial-aci with the same name already exists, please use a different service name or delete the existing service."
    }
}

## 7. Test the resulting web service.

When you deploy remotely, you may have key authentication enabled. The example below shows how to get your service key with Python in order to make an inference request.

In [11]:
import requests
import json
from azureml.core import Webservice

service = Webservice(workspace=ws, name='product-recommender-tutorial-aci')
scoring_uri = service.scoring_uri
print('scoring uri: ', scoring_uri)

# If the service is authenticated, set the key or token
primary_key, _ = service.get_keys()
print('key:', primary_key)

# Set the appropriate headers
headers = {'Content-Type': 'application/json'}
headers['Authorization'] = f'Bearer {primary_key}'

data_stringified = json.dumps(data)
response = requests.post(scoring_uri, data=data_stringified, headers=headers)
print('the model responded:')
print(response.text)
print(response.json())
print('time taken:', response.elapsed.total_seconds())


scoring uri:  http://4c57df6a-6624-4fb4-91ba-c02299f59ed5.westus2.azurecontainer.io/score
key: K64jyJGFpKxSSHEkxHwFAXigwd2nCBY7
the model responded:
{"productId": 5}
{'productId': 5}
time taken: 0.369844


You can get the logs from the remote ACI

In [13]:
print(service.get_logs())

2021-07-05T01:33:31,477563800+00:00 - rsyslog/run 
2021-07-05T01:33:31,482254700+00:00 - gunicorn/run 
2021-07-05T01:33:31,489107300+00:00 - iot-server/run 
2021-07-05T01:33:31,527425200+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_abe832532d68e7634ad48db5ac241baf/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_abe832532d68e7634ad48db5ac241baf/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_abe832532d68e7634ad48db5ac241baf/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_abe832532d68e7634ad48db5ac241baf/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_abe832532d68e7634ad48db5ac241baf/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
Starting

## Required information in Four2

In [105]:
print('scoring url:', scoring_uri)
print('key', primary_key)

scoring url: http://9a084848-7f0b-4a8e-85f1-84719493df60.westus2.azurecontainer.io/score
key cWAz6IhpSMeNesNnNCsQhglQlG6NdGn5


## WELL DONE!

You deployed your first model!

The next part is making it intelligent.

But first, clean up the resources you created.

In [69]:
# delete the ACI service you created
service.delete()

Container has been successfully cleaned up.


In [136]:
# delete all the models we created
models = Model.list(ws, name=model_parameters['name'])
for m in models:
    print('deleting', model.name, 'version:' , model.version)
    m.delete()